# Handle the CIFAR-10 dataset

In [33]:
%matplotlib inline
import pickle
import numpy as np
from os import listdir
from os.path import isfile, join
import os
from sklearn.preprocessing import StandardScaler

In [34]:
# Function to unpickle the dataset
def unpickle_all_data(directory):
    
    # Initialize the variables
    train = dict()
    test = dict()
    train_x = []
    train_y = []
    test_x = []
    test_y = []
    
    # Iterate through all files that we want, train and test
    # Train is separated into batches
    for filename in listdir(directory):
        if isfile(join(directory, filename)):
            
            # The train data
            if 'data_batch' in filename:
                print('Handing file: %s' % filename)
                
                # Opent the file
                with open(directory + '/' + filename, 'rb') as fo:
                    data = pickle.load(fo, encoding='bytes')

                if 'data' not in train:
                    train['data'] = data[b'data']
                    train['labels'] = np.array(data[b'labels'])
                else:
                    train['data'] = np.concatenate((train['data'], data[b'data']))
                    train['labels'] = np.concatenate((train['labels'], data[b'labels']))
            # The test data
            elif 'test_batch' in filename:
                print('Handing file: %s' % filename)
                
                # Open the file
                with open(directory + '/' + filename, 'rb') as fo:
                    data = pickle.load(fo, encoding='bytes')
                
                test['data'] = data[b'data']
                test['labels'] = data[b'labels']
    
    # Manipulate the data to the propper format
    for image in train['data']:
        train_x.append(np.transpose(np.reshape(image,(3, 32,32)), (1,2,0)))
    train_y = [label for label in train['labels']]
    
    for image in test['data']:
        test_x.append(np.transpose(np.reshape(image,(3, 32,32)), (1,2,0)))
    test_y = [label for label in test['labels']]
    
    # Transform the data to np array format
    train_x = np.array(train_x)
    train_y = np.array(train_y)
    test_x = np.array(test_x)
    test_y = np.array(test_y)
    
    return (train_x, train_y), (test_x, test_y)

In [35]:
(x_train, y_train), (x_test, y_test) = unpickle_all_data(os.getcwd() + '/cifar-10-batches-py/')

Handing file: data_batch_1
Handing file: data_batch_2
Handing file: data_batch_3
Handing file: data_batch_4
Handing file: data_batch_5
Handing file: test_batch


In [36]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(50000, 32, 32, 3)
(50000,)
(10000, 32, 32, 3)
(10000,)


In [37]:
from keras.utils import np_utils

# Transofrm them to a float32 type
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize the input 
x_train /= 255
x_test /= 255

# One-hot Encoding
num_classes = 10
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)

In [38]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(50000, 32, 32, 3)
(50000, 10)
(10000, 32, 32, 3)
(10000, 10)


In [39]:
import keras
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar10
from keras import regularizers
from keras.callbacks import LearningRateScheduler
import numpy as np

# Create the model
weight_decay = 1e-4
model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

In [40]:
#data augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    )
datagen.fit(x_train)

In [41]:
# Compile the model
batch_size = 64

opt_rms = keras.optimizers.rmsprop(lr=0.001, decay=1e-6)
model.compile(loss='categorical_crossentropy', 
              optimizer=opt_rms, 
              metrics=['accuracy'])

In [42]:
epochs = 100
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=x_train.shape[0] // batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

Epoch 1/100
781/781 [==============================] - 28s 36ms/step - loss: 1.9264 - acc: 0.4339 - val_loss: 1.9785 - val_acc: 0.4608
Epoch 2/100
781/781 [==============================] - 26s 33ms/step - loss: 1.4268 - acc: 0.5939 - val_loss: 1.4659 - val_acc: 0.5449
Epoch 3/100
781/781 [==============================] - 25s 32ms/step - loss: 1.2928 - acc: 0.6386 - val_loss: 1.0575 - val_acc: 0.6661
Epoch 4/100
781/781 [==============================] - 25s 32ms/step - loss: 1.1501 - acc: 0.6741 - val_loss: 0.9429 - val_acc: 0.7120
Epoch 5/100
781/781 [==============================] - 25s 32ms/step - loss: 1.1266 - acc: 0.6917 - val_loss: 0.9799 - val_acc: 0.7139
Epoch 6/100
781/781 [==============================] - 25s 32ms/step - loss: 1.0450 - acc: 0.7146 - val_loss: 0.9205 - val_acc: 0.7292
Epoch 7/100
781/781 [==============================] - 25s 32ms/step - loss: 1.0558 - acc: 0.7218 - val_loss: 0.9595 - val_acc: 0.7374
Epoch 8/100
781/781 [==============================] - 

781/781 [==============================] - 26s 33ms/step - loss: 0.6669 - acc: 0.8292 - val_loss: 0.6998 - val_acc: 0.8302
Epoch 62/100
781/781 [==============================] - 25s 33ms/step - loss: 0.6584 - acc: 0.8322 - val_loss: 0.7070 - val_acc: 0.8193
Epoch 63/100
781/781 [==============================] - 25s 33ms/step - loss: 0.6559 - acc: 0.8331 - val_loss: 0.7513 - val_acc: 0.8049
Epoch 64/100
781/781 [==============================] - 27s 34ms/step - loss: 0.6437 - acc: 0.8337 - val_loss: 0.6226 - val_acc: 0.8464
Epoch 65/100
781/781 [==============================] - 26s 33ms/step - loss: 0.6519 - acc: 0.8322 - val_loss: 0.6845 - val_acc: 0.8297
Epoch 66/100
781/781 [==============================] - 26s 34ms/step - loss: 0.6413 - acc: 0.8357 - val_loss: 0.7091 - val_acc: 0.8183
Epoch 67/100
781/781 [==============================] - 27s 34ms/step - loss: 0.6370 - acc: 0.8360 - val_loss: 0.6495 - val_acc: 0.8391
Epoch 68/100
781/781 [==============================] - 27s 3